Initial hacking on the STEW-MAP attributes.  Most of this is incorporated in notebooks above.

In [ ]:
import fiona

In [ ]:
borough_gdf = gpd.read_file('../data/admin-boundaries/Borough Boundaries.zip', dtypes={'boro_code': 'Int8'})

In [ ]:
borough_gdf.info()

In [ ]:
borough_gdf['boro_code'] = borough_gdf['boro_code'].astype('Int8')

In [ ]:
borough_gdf.info()

In [ ]:
borough_gdf.explore(column='boro_name', 
                    cmap=["red", "blue", "green", "yellow", "purple"])

In [ ]:
brooklyn_gdf = borough_gdf.loc[borough_gdf['boro_code'] ==  3]

In [ ]:
brooklyn_gdf.crs

In [ ]:
fiona.listlayers('../data/NYC-2017-STEW-MAP-Public-Version2/NYC2017_STEWMAP_Version2.gdb/')

In [ ]:
%%time
polygon_locations_gdf = gpd.read_file('../data/NYC-2017-STEW-MAP-Public-Version2/NYC2017_STEWMAP_Version2.gdb/',
                           driver='FileGDB',
                           dtypes={'DR_CC': 'Int64',
                                  'Dr_NeighDe': 'Int64',
                                  'Dr_ExtrWth': 'Int64'},
                           layer='NYC2017_STEWMAP_PublicPolygons_Version2')

In [ ]:
polygon_locations_gdf.PopID.is_unique

In [ ]:
polygon_locations_gdf.PopID.value_counts()

In [ ]:
dup_popid = [20212, 1190, 20333]

In [ ]:
nondup_turfs = polygon_locations_gdf.query(f"PopID not in @dup_popid")

In [ ]:
dup_turfs['OrgName']

In [ ]:
nondup_turfs.PopID.is_unique

In [ ]:
%%time
point_locations_gdf = gpd.read_file('../data/NYC-2017-STEW-MAP-Public-Version2/NYC2017_STEWMAP_Version2.gdb/',
                           driver='FileGDB',
                           dtypes={'DR_CC': 'Int64',
                                  'Dr_NeighDe': 'Int64',
                                  'Dr_ExtrWth': 'Int64'},
                           layer='NYC2017_STEWMAP_PublicPoints_Version2')

In [ ]:
point_locations_gdf.PopID.value_counts()

In [ ]:
in_brooklyn_gdf = brooklyn_gdf.to_crs('epsg:4326').overlay(polygon_locations_gdf.to_crs('epsg:4326'), how='intersection')

In [ ]:
list(in_brooklyn_gdf.columns)

In [ ]:
in_brooklyn_gdf.loc[in_brooklyn_gdf['OrgName'] != 'FABSCRAP'].explore(
    column='OrgName', 
    tooltip=['OrgName', 'OrgWebSite', 'OrgTwitter', 'boro_name'],
    popup=True, 
    legend=False,
    width=1000,
    tiles="CartoDB positron", 
    cmap="Set1", 
    style_kwds=dict(color="black") 
    )

In [ ]:
%%time
office_locations_gdf = gpd.read_file('../data/NYC-2017-STEW-MAP-Public-Version2/NYC2017_STEWMAP_Version2.gdb/',
                           driver='FileGDB',
                           dtypes={'DR_CC': 'Int64',
                                  'Dr_NeighDe': 'Int64',
                                  'Dr_ExtrWth': 'Int64'},
                           layer='NYC2017_STEWMAP_PublicPoints_Version2')

#office_locations_gdf['DR_CC'] = office_locations_gdf.DR_CC.astype('Int64')

#office_locations_gdf['Dr_NeighDe'] = office_locations_gdf.Dr_NeighDe.astype('Int64')

#office_locations_gdf['Dr_ExtrWth'] = office_locations_gdf.Dr_ExtrWth.astype('Int64')

In [ ]:
office_locations_gdf.info(verbose=True, show_counts=True)

In [ ]:
office_locations_gdf[:20][['OrgName', 'OrgWebSite']]

In [ ]:
empty_url = ['', ' ']

In [ ]:
foo = office_locations_gdf.query(f"OrgWebSite in @empty_url")

In [ ]:
bar = office_locations_gdf.query(f"OrgWebSite not in @empty_url").reset_index().drop(columns=['index'])

In [ ]:
bar.info()

In [ ]:
list(bar[:40]['OrgWebSite'])

In [ ]:
office_locations_gdf.iloc[27]['DR_CC'] == 7

In [ ]:
bar['OrgZip'].value_counts()

In [ ]:
bar.iloc[27]['OrgZip'][:3]

In [ ]:
bar['zip_prefix'] = bar['OrgZip'].apply(lambda v: v[:3])

In [ ]:
brooklyn = bar.loc[bar['zip_prefix'] == '112']

In [ ]:
brooklyn['OrgZip'].value_counts()

In [ ]:
%%time
turfs_gdf = gpd.read_file('../data/NYC-2017-STEW-MAP-Public-Version2/NYC2017_STEWMAP_Version2.gdb/',
                           driver='FileGDB',
                           layer='NYC2017_STEWMAP_PublicPolygons_Version2')

In [ ]:
office_columns = set(office_locations_gdf)
turf_columns = set(turfs_gdf)


In [ ]:
len(office_columns)

In [ ]:
len(turf_columns)

In [ ]:
office_columns.difference(turf_columns)

In [ ]:
turf_columns.difference(office_columns)

So, as stated in office hours the only differences are in geometries.

Not sure about ResID_1 ...

In [ ]:
columns_in_both = list(office_columns.intersection(turf_columns))

In [ ]:
turfs_both_gdf = turfs_gdf[columns_in_both]
office_locations_both_gdf = office_locations_gdf[columns_in_both]

In [ ]:
office_locations_gdf.ResID_1.value_counts()

In [ ]:
office_locations_gdf.info(verbose=True, show_counts=True)

In [ ]:
office_locations_gdf.DR_CC.value_counts()

In [ ]:
highly_influential = [7, 6]

In [ ]:
type(_)

In [ ]:
office_locations_gdf.DR_CC.value_counts()

In [ ]:
office_locations_gdf.PopID.is_unique

In [ ]:
list(office_locations_gdf.columns)

In [ ]:
cc_gdf = office_locations_gdf.query(f"DR_CC == @highly_influential").reset_index().drop(columns=['index'])

In [ ]:
cc_gdf.DR_CC.value_counts()

In [ ]:
zoning_gdf = office_locations_gdf.query(f"Dr_NeighDe == @highly_influential").reset_index().drop(columns=['index'])

In [ ]:
zoning_gdf.Dr_NeighDe.value_counts()

In [ ]:
extreme_gdf = office_locations_gdf.query(f"Dr_ExtrWth == @highly_influential").reset_index().drop(columns=['index'])

In [ ]:
extreme_gdf.info(verbose=True, show_counts=True)

In [ ]:
office_locations_gdf.Dr_NeighDe.value_counts()

In [ ]:
office_locations_gdf.Dr_ExtrWth.value_counts()

In [ ]:
s1 = set(cc__gdf.OBJECTID)

In [ ]:
s2 = set(zoning_gdf.OBJECTID)
s3 = set(extreme_gdf.OBJECTID)

In [ ]:
foo = set.intersection(s1, s2, s3)

In [ ]:
len(foo)